In [ ]:
import intake
import xarray as xr

df = intake.open_csv('./data/bird_migration/{species}.csv').read()
air = xr.open_dataset("./data/air.nc").air

In [ ]:
air = air.sel(lon=slice(205, 310), lat=slice(75, -55))

## Panel 

Panel provides the framework and interactivity to make dashboards that work in the notebook and can be deployed as standalone apps.

In [ ]:
import panel as pn

import hvplot.xarray
import hvplot.pandas

In [ ]:
day_slider = pn.widgets.Player(start=1, end=365, step=7, loop_policy='loop')
species_selector = pn.widgets.Select(options=list(df.species.cat.categories))

In [ ]:
bird_map = (
    air.hvplot("lon", "lat", groupby="day", geo=True, cmap="coolwarm", clim=(-40, 40), coastline=True, height=600) * \
    df.hvplot.points("lon", "lat", groupby="day", geo=True, color="white", title="Bird Location and Temperature") * \
    df.hvplot.points("lon", "lat", groupby=["day", "species"], geo=True, color="black", size=200, title="Bird Location and Temperature")
)

In [ ]:
bird_lon_lat = df.hvplot.points('lon', 'lat', color='day', cmap='colorwheel', groupby='species', title="")
bird_day_lat = df.hvplot('day', 'lat', groupby='species').hist().opts(title="")

In [ ]:
title = pn.pane.Markdown("### Overview of year", width=600)

def callback(target, event):
    target.object = f'### Overview of year for: {event.new.replace("_", " ")}'

species_selector.link(title, callbacks={'value': callback})

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Bird Migration', 
    sidebar=[
        pn.pane.Markdown("# Bird Migration"), 
        pn.pane.Markdown("Explore bird data collected by citizen scientists and cleaned and compiled by scientists. This data tracks the migratory patterns of various bird species over the course of a typical year. The data are coordinates of the centroid of different bird species with for each day of the year."),
        pn.pane.Markdown("## Settings"),   
        species_selector,
        pn.pane.Markdown("## References"), 
        pn.pane.Markdown('La Sorte FA, Fink D, Hochachka WM, Kelling S. 2016 "Convergence of broad-scale migration strategies in terrestrial birds". Proc. R. Soc. B 283: 20152588. http://dx.doi.org/10.1098/rspb.2015.2588'),
        pn.pane.Markdown('La Sorte FA, and Fink D. 2017 “Projected changes in prevailing winds for transatlantic migratory birds under global warming”. Journal of American Ecology 2017.86: 273-284. doi: 10.1111/1365-2656.12624'),
    ],
    main=[
        pn.Row(
            pn.Column(
                pn.Pane(bird_map, widgets={'species': species_selector, "day": day_slider}),
                pn.Pane(day_slider),
            ),
            pn.Column(
                title,
                pn.Pane(bird_lon_lat, widgets={'species': species_selector}, sizing_mode='stretch_width'),
                pn.Pane(bird_day_lat, widgets={'species': species_selector}, sizing_mode='stretch_width'),
            ),
        )
    ],
    accent_base_color="#0479cc",
    header_background="#0479cc",
)
template.show()